In [28]:
import pandas as pd
import numpy as np


from core import constants
from core.utils import *

log = get_logger()

pd.set_option("display.float_format", "{:.2f}".format)

## RPGF 3 Data Check and Cleanup

In [29]:
df = pd.read_csv("data/dummy_data_rpgf3.csv")

In [30]:
display(df.sample(10))

,Address,Has voted,Has published,Published at,Created at,Updated at,Projects in ballot,Votes
684,mKaRKw3knH,True,True,2023-11-16 18:54:05,2023-09-15 19:39:19,2023-09-29 04:27:24,11,"[{'amount': '25655', 'projectId': 'proj165'}, ..."
534,7bZqKs7d0x,True,True,2023-11-19 14:50:04,2023-10-22 03:38:09,2023-11-04 03:20:13,18,"[{'amount': '242734', 'projectId': 'proj257'},..."
94,LlpIn8srmR,False,False,NaN,2023-11-05 14:53:16,2023-11-18 10:53:31,0,[]
579,owFaLyeoEc,True,True,2023-11-03 06:51:41,2023-10-01 21:49:29,2023-10-14 20:57:48,11,"[{'amount': '364716', 'projectId': 'proj459'},..."
812,qijJcifY1T,False,False,NaN,2023-10-25 13:39:04,2023-11-25 01:00:53,0,[]
181,9PObQzxHDj,True,True,2023-11-30 23:46:25,2023-11-28 01:13:00,2023-11-30 20:04:42,13,"[{'amount': '746652', 'projectId': 'proj9'}, {..."
623,xjT0UXNX8a,True,False,NaN,2023-10-02 20:09:53,2023-11-12 19:32:41,0,[]
596,RJKVjrnwws,False,False,NaN,2023-11-16 10:42:51,2023-11-27 09:04:38,0,[]
271,bJu4Xej0vx,False,False,NaN,2023-10-24 21:39:23,2023-11-19 22:00:29,0,[]
631,cyXUA4nlNu,False,False,NaN,2023-10-26 15:59:25,2023-11-21 01:03:21,0,[]


In [31]:
log.info("Check - Num Ballots: " + str(df["Has published"].count()))
log.info("Check - Num Submissions (Published): " + str(df["Has published"].sum()))

# Check if voter_address is unique
if df["Address"].nunique() == df.shape[0]:
    log.info("Check - Address is unique.")
else:
    diff = df.shape[0] - df["Address"].nunique()
    log.info(f"Check - Address is not unique. There are {diff} duplicates.")

# Check if all voters have voted
if df[df["Has voted"] == False].shape[0] > 0:
    not_voted = df[df["Has voted"] == False].shape[0]
    total = df["Address"].nunique()
    log.info(f"Check - {not_voted} voters out of {total} have not voted.")
else:
    log.info("Check - All voters have voted.")

# Check if all voters have published
if df[df["Has published"] == False].shape[0] > 0:
    not_voted = df[df["Has published"] == False].shape[0]
    total = df["Address"].nunique()
    log.info(f"Check - {not_voted} voters out of {total} have not published.")
else:
    log.info("Check - All voters have published.")

2024-01-03 18:36:09 INFO | Check - Num Ballots: 1000
2024-01-03 18:36:09 INFO | Check - Num Submissions (Published): 514
2024-01-03 18:36:09 INFO | Check - Address is unique.
2024-01-03 18:36:09 INFO | Check - 243 voters out of 1000 have not voted.
2024-01-03 18:36:09 INFO | Check - 486 voters out of 1000 have not published.


In [32]:
# Apply the function and concatenate results
expanded_list = [
    expand_json(safe_json_loads(row), idx) for idx, row in df["Votes"].items()
]
expanded_df = pd.concat(expanded_list, ignore_index=True)

result_df = expanded_df.set_index("original_index").join(df.set_index(df.index))

In [33]:
testing_address = "zgdSu8Yr87"
print_df = result_df[result_df["Address"] == testing_address]
print("Num Projects Voted : " + str(print_df["projectId"].count()))
display(print_df.head(10))

Num Projects Voted : 18


,amount,projectId,Address,Has voted,Has published,Published at,Created at,Updated at,Projects in ballot,Votes
830,703749,proj117,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,385034,proj51,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,470511,proj230,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,113619,proj204,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,156327,proj141,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,622540,proj390,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,32962,proj482,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,911617,proj46,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,755757,proj250,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."
830,114436,proj82,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,"[{'amount': '703749', 'projectId': 'proj117'},..."


In [34]:
columns = [col for col in result_df.columns if col not in ["amount", "projectId"]]
columns += ["amount", "projectId"]  # Add the columns to the end of the list
result_df = result_df[columns]

# Update df columns names
result_df.columns = [
    "voter_address",
    "has_voted",
    "has_published",
    "published_at",
    "created_at",
    "updated_at",
    "projects_in_ballot",
    "votes",
    "amount",
    "project_id",
]

result_df.drop(columns="votes", inplace=True)

result_df["amount"] = pd.to_numeric(result_df["amount"])

In [35]:
# result_df.head()
result_df[result_df["voter_address"] == testing_address].head(70)

,voter_address,has_voted,has_published,published_at,created_at,updated_at,projects_in_ballot,amount,project_id
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,703749,proj117
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,385034,proj51
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,470511,proj230
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,113619,proj204
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,156327,proj141
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,622540,proj390
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,32962,proj482
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,911617,proj46
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,755757,proj250
830,zgdSu8Yr87,True,True,2023-11-02 18:18:29,2023-09-21 02:51:22,2023-09-23 16:56:39,18,114436,proj82


## Calculate Voting Results

In [36]:
allocator = ProjectAllocator(
    total_amount=constants.TOTAL_AMOUNT,
    min_amount=constants.MIN_AMOUNT,
    quorum=constants.QUORUM,
)

In [37]:
initial_allocation = allocator.calculate_initial_allocation(result_df)

In [38]:
display(initial_allocation.sample(1))

,votes_count,median_amount,is_eligible
project_id,,,
proj178,12,493485.50,False


In [39]:
# Scaling the total to 30M OP by project and filter out those with < 1500 OP
allocation_iter = initial_allocation[initial_allocation["is_eligible"] == True].copy()
allocation_iter["scaled_amount"] = allocation_iter["median_amount"]
# display(allocation_iter)
# Set a maximum number of iterations to prevent infinite loop
max_iterations = 10
current_iteration = 0

while (
    allocation_iter["scaled_amount"].sum() != constants.TOTAL_AMOUNT
    and current_iteration <= max_iterations
):
    allocation_iter = allocator.scale_allocations_oneby(allocation_iter)
    current_iteration += 1

    log.info("Check - Current iteration: " + str(current_iteration))

2024-01-03 18:36:10 INFO | Check - Original Amount Eligible: 12322787.0
2024-01-03 18:36:10 INFO | Check - Scale Factor: 2.434514205268662
2024-01-03 18:36:10 INFO | Check - No projects below minimum OP
2024-01-03 18:36:10 INFO | Check - Current iteration: 1


In [40]:
# Check if the loop exited due to reaching max iterations
if (
    current_iteration == max_iterations
    and allocation_iter["scaled_amount"].sum() != constants.TOTAL_AMOUNT
):
    log.info("Maximum iterations reached without meeting the total amount condition.")
else:
    final_total = allocation_iter["scaled_amount"].sum()
    log.info(
        f"Condition met with {final_total} OP allocated through {current_iteration} iteration(s)."
    )

2024-01-03 18:36:10 INFO | Condition met with 30000000.0 OP allocated through 1 iteration(s).


In [41]:
# join the initial allocation with the final allocation, if scaled_amount is null then make it 0
final_allocation = initial_allocation.merge(
    allocation_iter["scaled_amount"],
    how="left",
    on="project_id",
).fillna({"scaled_amount": 0})

# check if the final allocation table still contains all projects.
if final_allocation.index.nunique() == result_df["project_id"].nunique():
    log.info("Check - Final allocation table has included all the projects.")
else:
    log.info(
        "Check - Final allocation table has missing projects. Printing out the missing projects below."
    )
    log.info(
        result_df[~result_df["project_id"].isin(final_allocation.index)]["project_id"]
    )

# check if the final allocation table still sums to the total amount.
if final_allocation["scaled_amount"].sum() == final_total:
    log.info(
        "Check - Final allocation table sums to the right amount of OP: "
        + str(final_total)
    )
else:
    log.info(
        "Check - Final allocation table does not sum to the total OP. Printing out the missing amount below."
    )
    log.info(str(final_total - final_allocation["scaled_amount"].sum()) + " OP")

2024-01-03 18:36:10 INFO | Check - Final allocation table has included all the projects.
2024-01-03 18:36:10 INFO | Check - Final allocation table sums to the right amount of OP: 30000000.0


In [42]:
# export csv
# allocation_iter.drop(columns="median_amount", inplace=True)
final_allocation.to_csv("data/rpgf3_allocation_final.csv")

log.info(f"Results saved in data/rpgf3_allocation_final.csv.")

2024-01-03 18:36:10 INFO | Results saved in data/rpgf3_allocation_final.csv.


In [43]:
final_allocation.head(10)

,votes_count,median_amount,is_eligible,scaled_amount
project_id,,,,
proj26,5,841223.00,False,0.00
proj323,11,838886.00,False,0.00
proj123,7,829909.00,False,0.00
proj170,10,817377.00,False,0.00
proj46,11,802627.00,False,0.00
proj269,10,793657.00,False,0.00
proj12,11,787796.00,False,0.00
proj196,7,782668.00,False,0.00
proj287,6,782357.00,False,0.00


In [44]:
to_cut = (
    allocation_iter[allocation_iter["scaled_amount"] < 1500]
    .sort_values(by="scaled_amount")
    .head(1)
)

In [45]:
# check if to_cut is empty
to_cut.empty

True

### Calculate Voting Results using pytorch

In [46]:

result_tensor, num_projects = allocator.convert_df_to_tensor(result_df)
project_tensors = allocator.get_project_tensor(result_tensor, num_projects)


voter_address         object
has_voted               bool
has_published           bool
published_at          object
created_at            object
updated_at            object
projects_in_ballot     int64
amount                 int64
project_id            object
dtype: object
     voter_address  has_voted  has_published         published_at  \
689            491       True           True  2023-11-03 02:22:07   
897            240       True           True  2023-11-21 01:16:47   
790            260       True           True  2023-11-25 17:02:30   
643            156       True           True  2023-11-25 13:23:26   
160             74       True           True  2023-11-20 19:47:08   

              created_at           updated_at  projects_in_ballot  amount  \
689  2023-10-31 03:01:09  2023-11-02 13:18:21                   9   57138   
897  2023-11-17 09:56:08  2023-11-17 14:52:13                  15  199228   
790  2023-11-22 11:35:04  2023-11-24 11:25:56                   3  230502   
64

In [47]:
# export to onnx
allocator.eval()

# convert projects tensors to tuple
final_allocation_torch = allocator.forward(*project_tensors)
final_allocation.shape

input_names = ['input_' + str(i) for i in range(len(project_tensors))]
    # Export the model
torch.onnx.export(allocator,               # model being run
                      tuple(project_tensors),          # model input (or a tuple for multiple inputs)
                      "network.onnx",           # where to save the model (can be a file or file-like object)
                      export_params=False,       # store the trained parameter weights inside the model file
                      opset_version=17,         # the ONNX version to export the model to
                      do_constant_folding=True, # whether to execute constant folding for optimization
                      input_names = input_names,   # the model's input names
                      output_names = ['output'])

/Users/alexandercamuto/Documents/GitHub/zk-rpgf/core/utils.py:165: TracerWarning:

torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.



We now include sanity checks for the data. We will check the following:
- that the pandas and pytorch dataframes are the same
- that the median and scaled median allocations are the same
- that eligibility is the same

In [48]:

# a bunch of code to compare pytorch output and the pandas output as a sanity check
final_allocation_torch_np = final_allocation_torch.detach().numpy()
final_allocation_torch_df = pd.DataFrame(final_allocation_torch_np, columns=["votes_count", "median_amount", "is_eligible", "scaled_amount"])
# convert is eligible to boolean
final_allocation_torch_df["is_eligible"] = final_allocation_torch_df["is_eligible"].astype(bool)
# convert votes count to int
final_allocation_torch_df["votes_count"] = final_allocation_torch_df["votes_count"].astype(int)
# add index project_id
final_allocation_torch_df = final_allocation_torch_df.set_index(final_allocation.index.sort_values())

# sorted vy project id
final_allocation_torch_df = final_allocation_torch_df.sort_index()
# sort final allocation by project id
final_allocation_sorted = final_allocation.sort_index()

display(final_allocation_torch_df.head(10))
display(final_allocation_sorted.head(10))
# compare the two outputs
print("Are the two outputs equal?")
final_allocation_torch_df.compare(final_allocation_sorted)


,votes_count,median_amount,is_eligible,scaled_amount
project_id,,,,
proj0,8,408033.00,False,0.00
proj1,10,462152.00,False,0.00
proj10,13,398607.00,False,0.00
proj100,9,259604.00,False,0.00
proj101,9,379887.00,False,0.00
proj102,13,460381.00,False,0.00
proj103,13,526428.00,False,0.00
proj104,10,350640.00,False,0.00
proj105,4,538145.50,False,0.00


,votes_count,median_amount,is_eligible,scaled_amount
project_id,,,,
proj0,8,408033.00,False,0.00
proj1,10,462152.00,False,0.00
proj10,13,398607.00,False,0.00
proj100,9,259604.00,False,0.00
proj101,9,379887.00,False,0.00
proj102,13,460381.00,False,0.00
proj103,13,526428.00,False,0.00
proj104,10,350640.00,False,0.00
proj105,4,538145.50,False,0.00


Are the two outputs equal?


scaled_amount           
                    self      other
project_id                         
proj109       1336049.38 1312724.14
proj167       1478419.62 1452608.89
proj174       1676150.75 1646888.00
proj209       1833628.00 1801615.98
proj232        929028.25  912808.93
proj242       1256213.12 1234281.66
proj275       1379720.12 1355632.46
proj303       1286749.12 1264284.61
proj309       1076575.88 1057780.60
proj315        844697.31  829950.24
proj356        945747.00  929235.81
proj390       1243804.38 1222089.61
proj407       1252619.12 1230750.40
proj413        945123.88  928623.53
proj427        973818.88  956817.64
proj447       1002437.19  984936.28
proj451       1143725.88 1123758.37
proj470       1735069.75 1704778.31
proj484       1362222.12 1338439.92
proj489       1595328.38 1567476.58
proj499       1422662.38 1397825.02
proj51        1093450.75 1074360.86
proj59        1204028.62 1183008.36
proj71        1515787.00 1489323.80

TODO: investigate the small differences in the scaled median allocations. Seems like pandas adds some small epsilons during calculations that compound.

After which we can proceed to generate the settings file for `ezkl` and run calibrate settings to find the optimal settings for `ezkl`

In [49]:
import ezkl
import os

model_path = "network.onnx"
settings_path = "settings.json"
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default
py_run_args.num_inner_cols = 1

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True

We calibrate the settings to finetune the circuit to the data. This is done by running the `calibrate_settings` function.

In [50]:
data_path = os.path.join("input.json")

data = dict(input_data = [tensor.detach().numpy().reshape([-1]).tolist() for tensor in project_tensors])

# Serialize data into file:
json.dump(data, open(data_path, 'w'))

ezkl.calibrate_settings(data_path, model_path, settings_path, "accuracy", lookup_safety_margin=1, scales=[14])



max lookup input (558195204096000) is too large
max lookup input (8045347637035008000) is too large


True

In [51]:
# load settings and shrink run_args.logrows by 1 to induce overflow for lookups 
settings = json.load(open(settings_path, 'r'))
settings['run_args']['logrows'] -= 1
json.dump(settings, open(settings_path, 'w'))

Next, we will compile the model. The compilation step allow us to generate proofs faster.

In [52]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

Before we can setup the circuit params, we need a SRS (Structured Reference String). The SRS is used to generate the proofs.

In [53]:
# srs path
res = ezkl.get_srs( settings_path)

In [54]:
# now generate the witness file 

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

Now run setup, this will generate a proving key (pk) and verification key (vk). The proving key is used for proving while the verification key is used for verificaton.

In [ ]:

# setup
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
spawning module 2
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
spawning module 2


In [ ]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
spawning module 2


: 

We can now verify the proof. 

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

assert res == True
print("verified")

We can now create an EVM / `.sol` verifier that can be deployed on chain to verify submitted proofs using a view function.

In [ ]:

abi_path = 'test.abi'
sol_code_path = 'test_1.sol'

res = ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True

# Sanity checks on circuit outputs

In [56]:

import onnx
import onnxruntime
import os
import json
import ezkl
import numpy as np

witness_path = os.path.join('witness.json')
settings_path = os.path.join('settings.json')
model_path = os.path.join('network.onnx')
data_path = os.path.join('input.json')

def get_ezkl_output(witness_file, settings_file):
    # convert the quantized ezkl output to float value
    witness_output = json.load(open(witness_file))
    outputs = witness_output['outputs']
    with open(settings_file) as f:
        settings = json.load(f)
    ezkl_outputs = [[ezkl.vecu64_to_float(
        outputs[i][j], settings['model_output_scales'][i]) for j in range(len(outputs[i]))] for i in range(len(outputs))]
    return ezkl_outputs


def get_onnx_output(model_file, input_file):
    # generate the ML model output from the ONNX file
    onnx_model = onnx.load(model_file)
    onnx.checker.check_model(onnx_model)

    with open(input_file) as f:
        inputs = json.load(f)
    # reshape the input to the model
    num_inputs = len(inputs['input_data'])

    onnx_input = dict()
    for i in range(num_inputs):
        input_node = onnx_model.graph.input[i]
        dims = []
        elem_type = input_node.type.tensor_type.elem_type
        for dim in input_node.type.tensor_type.shape.dim:
            if dim.dim_value == 0:
                dims.append(1)
            else:
                dims.append(dim.dim_value)
        if elem_type == 7:
            inputs_onnx = np.array(inputs['input_data'][i]).astype(
                np.int64).reshape(dims)
        elif elem_type == 9:
            inputs_onnx = np.array(inputs['input_data'][i]).astype(
                bool).reshape(dims)
        else:
            inputs_onnx = np.array(inputs['input_data'][i]).astype(
                np.float32).reshape(dims)
        onnx_input[input_node.name] = inputs_onnx
    try:
        onnx_session = onnxruntime.InferenceSession(model_file)
        onnx_output = onnx_session.run(None, onnx_input)
    except Exception as e:
        print("Error in ONNX runtime: ", e)
        print("using inputs[output_data]")
        onnx_output = inputs['output_data']
    return onnx_output[0]


def compare_outputs(zk_output, onnx_output):
    # calculate hamming difference between the 2 outputs (which are lists)

    res = []
    
   

    contains_sublist = any(isinstance(sub, list) for sub in zk_output)
    if contains_sublist:
        try:
            if len(onnx_output) == 1:
                zk_output = zk_output[0]
        except Exception as e:
            zk_output = zk_output[0]

    flat_zk_output = np.array(zk_output).flatten()
    flat_onnx_output = np.array(onnx_output).flatten()
    
    
    # select every 4th element
    scaled_amount_zk = flat_zk_output[3::4]
    scaled_amount_onnx = flat_onnx_output[3::4]
    
    # print the diff where they are not both 0
    print("scaled amount diff")
    for i in range(len(scaled_amount_zk)):
        if scaled_amount_zk[i] != 0.0 and scaled_amount_onnx[i] != 0.0:
            print("scaled amount zk: ", scaled_amount_zk[i])
            print("scaled amount onnx: ", scaled_amount_onnx[i])
            print("diff: ", scaled_amount_zk[i] - scaled_amount_onnx[i])
            print("percent diff: ", 100*(scaled_amount_zk[i] - scaled_amount_onnx[i]) / scaled_amount_onnx[i])
    
    zip_object = zip(flat_zk_output[3::4], flat_onnx_output[3::4])
    
    for list1_i, list2_i in zip_object:
        if list1_i == 0.0 and list2_i == 0.0:
            res.append(0)
        else:
            percent_diff = (list1_i - list2_i) / list2_i
            res.append(percent_diff)


    return np.abs(res)

In [ ]:

import plotly.express as px
import pandas as pd

import matplotlib.pyplot as plt
    # get the ezkl output
ezkl_output = get_ezkl_output(witness_path, settings_path)
    # get the onnx output
onnx_output = get_onnx_output(model_path, data_path)
    # compare the outputs
l1_difference = compare_outputs(ezkl_output, onnx_output)

df = pd.DataFrame(l1_difference, columns=["percent error"])


# Create a histogram
fig = px.histogram(df, x="percent error",
                  title="Distribution of percent error")
fig.show()

scaled amount diff
scaled amount zk:  1332895.9350585938
scaled amount onnx:  1336049.4
diff:  -3153.43994140625
percent diff:  -0.23602720082154524
scaled amount zk:  1474930.2062988281
scaled amount onnx:  1478419.6
diff:  -3489.418701171875
percent diff:  -0.23602356476916186
scaled amount zk:  1672194.6716308594
scaled amount onnx:  1676150.8
diff:  -3956.078369140625
percent diff:  -0.23602163284779876
scaled amount zk:  1829300.2624511719
scaled amount onnx:  1833628.0
diff:  -4327.737548828125
percent diff:  -0.2360204768267132
scaled amount zk:  926835.4797363281
scaled amount onnx:  929028.25
diff:  -2192.770263671875
percent diff:  -0.23602837305236682
scaled amount zk:  1253248.0773925781
scaled amount onnx:  1256213.1
diff:  -2965.047607421875
percent diff:  -0.2360306184049681
scaled amount zk:  1376463.5925292969
scaled amount onnx:  1379720.1
diff:  -3256.532470703125
percent diff:  -0.23602848227665918
scaled amount zk:  1283712.0666503906
scaled amount onnx:  1286749.1